In [51]:
import pandas as pd
import features_processing_utils as fpu

In [55]:
username = "mjfree.bsky.social"
profile = fpu.get_profile(username)
profile

In [56]:
profile['followersCount']

271255

In [57]:
profile['followsCount']

2026

In [60]:
uri = "at://did:plc:gkgmduxh722ocstroyi75gbg/app.bsky.feed.post/3le56mzpymk2f"
post = fpu.get_post(uri)
post

{'posts': [{'uri': 'at://did:plc:gkgmduxh722ocstroyi75gbg/app.bsky.feed.post/3le56mzpymk2f',
   'cid': 'bafyreiazqnepgmtpwqq7e2m6jsjalkfprr2qexzfqacboa7slixgq3fesi',
   'author': {'did': 'did:plc:gkgmduxh722ocstroyi75gbg',
    'handle': 'mjfree.bsky.social',
    'displayName': 'Morgan J Freeman',
    'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gkgmduxh722ocstroyi75gbg/bafkreigb5of6ysnvijil2i2kci2nvxnctadvnyo3gjgq2usbaspdiz5yvq@jpeg',
    'labels': [],
    'createdAt': '2024-11-09T03:00:34.077Z'},
   'record': {'$type': 'app.bsky.feed.post',
    'createdAt': '2024-12-25T14:36:00.018Z',
    'langs': ['en'],
    'text': 'Merry Fucking Christmas 🎄'},
   'replyCount': 48,
   'repostCount': 38,
   'likeCount': 712,
   'quoteCount': 5,
   'indexedAt': '2024-12-25T14:36:00.248Z',
   'labels': []}]}

In [62]:
reposts = fpu.get_reposts(uri, limit=100)
reposts

{'repostedBy': [{'did': 'did:plc:zxn4wqkjmzbf5b3utmjebe2u',
   'handle': 'randomfurryfan.bsky.social',
   'displayName': 'Lesbian Furry Mayhem',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zxn4wqkjmzbf5b3utmjebe2u/bafkreidkvms2vms6uakyvfwop4kebn2awe26ttmkvpfm2gbiephutxfoa4@jpeg',
   'labels': [],
   'createdAt': '2024-11-21T22:59:05.135Z',
   'description': 'President Of Furkistan\n#1 Amy Rose Fan\nTikTok: www.tiktok.com/@furrystuffbecauseiwantto',
   'indexedAt': '2024-12-23T18:58:25.743Z'},
  {'did': 'did:plc:zn7lhrqjrbao6flovgbxwbup',
   'handle': 'amsterdamred-5280.bsky.social',
   'displayName': 'amsterdamred**NOT LOOKING FOR LOVE or Bitcoin or $$investment**',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zn7lhrqjrbao6flovgbxwbup/bafkreiew7knjasi34e5jwzlarpkqr7jysfcxszlqexljwv6bjsvsnw25da@jpeg',
   'labels': [],
   'createdAt': '2024-11-13T19:42:29.573Z',
   'description': 'I live a Mile HIGH 5280 here in Denver, Colorado, with my two dogs. Big Red 1

In [64]:
feed = fpu.get_feed("randomfurryfan.bsky.social", limit=100)
feed

{'feed': [{'post': {'uri': 'at://did:plc:meoglbda4wnbcleg4ncqbb2o/app.bsky.feed.post/3le6hgnyvis2x',
    'cid': 'bafyreib4lon6ih5aef3eihv3eqquanoaeavuiysbxr5575h5f4habemi64',
    'author': {'did': 'did:plc:meoglbda4wnbcleg4ncqbb2o',
     'handle': 'riverh56.bsky.social',
     'displayName': 'River H 💙🌊\U0001faf6',
     'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:meoglbda4wnbcleg4ncqbb2o/bafkreieup44guadrvv46tnzy7fvewwypgy2ip5mniabwij5azogsrtzwla@jpeg',
     'labels': [],
     'createdAt': '2024-11-23T23:28:32.646Z'},
    'record': {'$type': 'app.bsky.feed.post',
     'createdAt': '2024-12-26T02:46:09.813Z',
     'embed': {'$type': 'app.bsky.embed.images',
      'images': [{'alt': '',
        'aspectRatio': {'height': 480, 'width': 480},
        'image': {'$type': 'blob',
         'ref': {'$link': 'bafkreiewyehgbvlmetpbnirzr4ppch5o2kffn2fbfxakmx5nlo25gltvjm'},
         'mimeType': 'image/jpeg',
         'size': 99993}}]},
     'langs': ['en'],
     'text': "Let's stand toge

In [67]:
query = "macron trump"
posts = fpu.search_posts(query)
posts

KeyboardInterrupt: 

In [ ]:
politifact_fake = pd.read_csv("FakeNewsNet/politifact_fake.csv")
politifact_fake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [ ]:
politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         432 non-null    object
 1   news_url   428 non-null    object
 2   title      432 non-null    object
 3   tweet_ids  392 non-null    object
dtypes: object(4)
memory usage: 13.6+ KB


In [ ]:
dataset_politifact_fake = fpu.create_dataset(politifact_fake, 10)
dataset_politifact_fake.head()

  0%|          | 0/432 [00:00<?, ?it/s]

100%|██████████| 432/432 [10:21<00:00,  1.44s/it]


,post_uri,post_cid,type,date,news_id,like_count,repost_count,user_name,follower_count,follows_count
0,at://did:plc:b4ytafwctzq7dl3of433x7lz/app.bsky...,bafyreieo5pg7lrxr3nqgcicdld7x7t2go2vx7zozr7j7l...,post,2024-12-26T14:08:58.800Z,politifact14404,0,0,danielleschatten.bsky.social,32,55
1,at://did:plc:2otmwo2cntpk63ghqvss4szp/app.bsky...,bafyreidctc5eykabogyqamrllmrpy57orajrghukwdcoo...,post,2024-12-26T14:04:46.231Z,politifact14404,0,0,puppybell.bsky.social,307,108
2,at://did:plc:uaezc4lyaj7l34j37rsxrm4j/app.bsky...,bafyreiahtx7lix6u4umunipxep6j7id3b2pktysk6id4y...,post,2024-12-26T14:03:48.027Z,politifact14404,2,0,alanacct.bsky.social,578,723
3,at://did:plc:j7cf7ocetbiy77l6jnpu4g5d/app.bsky...,bafyreidsifrc4haod4vpa7j6khqeqygs76ix4nhssu5c2...,post,2024-12-26T14:00:29.928Z,politifact14404,0,0,queenbuttpirate.bsky.social,231,552
4,at://did:plc:7rd2my6qywq65ez5ke6jnlsn/app.bsky...,bafyreif3pzpa664ibpgyirkmzkmc4juhwuifesrqdwasn...,post,2024-12-26T13:57:04.848Z,politifact14404,0,0,writeroftruth.bsky.social,3959,7381


In [ ]:
dataset_politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        196 non-null    object
 1   post_cid        196 non-null    object
 2   type            196 non-null    object
 3   date            196 non-null    object
 4   news_id         196 non-null    object
 5   like_count      196 non-null    int64 
 6   repost_count    196 non-null    int64 
 7   user_name       196 non-null    object
 8   follower_count  196 non-null    int64 
 9   follows_count   196 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 15.4+ KB


In [25]:
politifact_real = pd.read_csv("FakeNewsNet/politifact_real.csv")

In [26]:
politifact_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         624 non-null    object
 1   news_url   567 non-null    object
 2   title      624 non-null    object
 3   tweet_ids  409 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [28]:
dataset_politifact_real = fpu.create_dataset(politifact_real, 10)

100%|██████████| 624/624 [20:51<00:00,  2.01s/it] 


In [ ]:
dataset_politifact_real.head()

,post_uri,post_cid,type,date,news_id,like_count,repost_count,user_name,follower_count,follows_count
0,at://did:plc:22g3mf3sm5z4lp7zfk6ncv32/app.bsky...,bafyreihsqwltxzvy55shly67exttzmamirfrrbgutaysm...,post,2024-12-26T00:33:20.223Z,politifact14984,0,0,prognews.bsky.social,86,12
1,at://did:plc:ngygrbhyzeyigjlxnli3r2xw/app.bsky...,bafyreiaim3w4bcbajxupci7vfnw67ozr4cmy72ttheimf...,post,2024-12-22T14:05:12.057Z,politifact14984,1,0,themoneycruncher.bsky.social,1053,102
2,at://did:plc:il7345tnmrhnzaufvgk5qfcx/app.bsky...,bafyreif2fveaw7subm7yfe6avtzkucglpvibgp2qols7o...,post,2024-12-12T21:35:44.118Z,politifact14984,1,0,srubenfeld.bsky.social,848,322
3,at://did:plc:sk44rhbmaca3ltsa4c6v6da2/app.bsky...,bafyreigvqyv6ih42pbugjdg3enkbidvu7etu3kstqxtwb...,post,2024-12-10T14:34:58Z,politifact14984,1,0,worldnewsguru.bsky.social,200,0
4,at://did:plc:vyytbbzsqq4ihfgpdlgyjwc3/app.bsky...,bafyreieh4m2ej3nw2s3amso4hxlw2kk6fqoblryw5ooh2...,post,2024-12-10T14:06:59Z,politifact14984,0,0,newsw0rld.bsky.social,38,19


In [ ]:
dataset_politifact_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        1222 non-null   object
 1   post_cid        1222 non-null   object
 2   type            1222 non-null   object
 3   date            1222 non-null   object
 4   news_id         1222 non-null   object
 5   like_count      1222 non-null   int64 
 6   repost_count    1222 non-null   int64 
 7   user_name       1222 non-null   object
 8   follower_count  1222 non-null   int64 
 9   follows_count   1222 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 95.6+ KB


In [34]:
politifact_fake_reposts = fpu.add_reposts(dataset_politifact_fake, 10)

100%|██████████| 196/196 [03:51<00:00,  1.18s/it]


In [35]:
politifact_fake_reposts.head()

,post_uri,post_cid,type,retweet_date,news_id,like_count,repost_count,user_name,follower_count,follows_count
0,at://did:plc:eyxe3kye7ceiqqg5allx5rhe/app.bsky...,bafyreib7jh4rm6r3rcxtfbhnij6agdccbf47pnwf44lx7...,repost,2023-12-31T18:10:08.345Z,politifact13949,29,2,craniatology.bsky.social,5468,446
1,at://did:plc:eyxe3kye7ceiqqg5allx5rhe/app.bsky...,bafyreib7jh4rm6r3rcxtfbhnij6agdccbf47pnwf44lx7...,repost,2023-12-31T18:10:08.345Z,politifact13949,29,2,treeaffinity.bsky.social,2301,1376
2,at://did:plc:eyxe3kye7ceiqqg5allx5rhe/app.bsky...,bafyreidk6lfa7bxjlcsipeb2iejqptmsgezyzkaeclfd4...,repost,2023-12-20T15:09:41.341Z,politifact13949,23,1,craniatology.bsky.social,5468,446
3,at://did:plc:ouhrn5moyxg3nr73oxoze7qz/app.bsky...,bafyreiecuhas6dmhxoxjlocipzxb5evwdcc2nyj2ubwkx...,repost,2023-10-16T19:41:36.758Z,politifact13949,16,1,luxomancer.bsky.social,5680,2791
4,at://did:plc:uzfhqujf4iq4iilhsetxvkfe/app.bsky...,bafyreidipstdohiz26r6booecdmrxnobroh672fx7e3qh...,repost,2024-12-24T14:21:59.962Z,politifact14548,8,4,twinsfan81.bsky.social,11852,9389


In [36]:
politifact_fake_reposts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        116 non-null    object
 1   post_cid        116 non-null    object
 2   type            116 non-null    object
 3   retweet_date    116 non-null    object
 4   news_id         116 non-null    object
 5   like_count      116 non-null    int64 
 6   repost_count    116 non-null    int64 
 7   user_name       116 non-null    object
 8   follower_count  116 non-null    int64 
 9   follows_count   116 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 9.2+ KB


In [37]:
politifact_fake_posts = pd.concat([dataset_politifact_fake, politifact_fake_reposts])

In [38]:
politifact_fake_posts.head()

,post_uri,post_cid,type,date,news_id,like_count,repost_count,user_name,follower_count,follows_count,retweet_date
0,at://did:plc:b4ytafwctzq7dl3of433x7lz/app.bsky...,bafyreieo5pg7lrxr3nqgcicdld7x7t2go2vx7zozr7j7l...,post,2024-12-26T14:08:58.800Z,politifact14404,0,0,danielleschatten.bsky.social,32,55,NaN
1,at://did:plc:2otmwo2cntpk63ghqvss4szp/app.bsky...,bafyreidctc5eykabogyqamrllmrpy57orajrghukwdcoo...,post,2024-12-26T14:04:46.231Z,politifact14404,0,0,puppybell.bsky.social,307,108,NaN
2,at://did:plc:uaezc4lyaj7l34j37rsxrm4j/app.bsky...,bafyreiahtx7lix6u4umunipxep6j7id3b2pktysk6id4y...,post,2024-12-26T14:03:48.027Z,politifact14404,2,0,alanacct.bsky.social,578,723,NaN
3,at://did:plc:j7cf7ocetbiy77l6jnpu4g5d/app.bsky...,bafyreidsifrc4haod4vpa7j6khqeqygs76ix4nhssu5c2...,post,2024-12-26T14:00:29.928Z,politifact14404,0,0,queenbuttpirate.bsky.social,231,552,NaN
4,at://did:plc:7rd2my6qywq65ez5ke6jnlsn/app.bsky...,bafyreif3pzpa664ibpgyirkmzkmc4juhwuifesrqdwasn...,post,2024-12-26T13:57:04.848Z,politifact14404,0,0,writeroftruth.bsky.social,3959,7381,NaN


In [ ]:
politifact_fake_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 312 entries, 0 to 115
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        312 non-null    object
 1   post_cid        312 non-null    object
 2   type            312 non-null    object
 3   date            196 non-null    object
 4   news_id         312 non-null    object
 5   like_count      312 non-null    int64 
 6   repost_count    312 non-null    int64 
 7   user_name       312 non-null    object
 8   follower_count  312 non-null    int64 
 9   follows_count   312 non-null    int64 
 10  retweet_date    116 non-null    object
dtypes: int64(4), object(7)
memory usage: 29.2+ KB


In [72]:
politifact_fake_posts.to_csv("data/politifact_fake_posts.csv", index=False)

In [103]:
def get_features(dataframe, label):
    features_df = pd.DataFrame()
    
    dataframe["date"] = pd.to_datetime(dataframe["date"], format='ISO8601')
    dataframe["retweet_date"] = pd.to_datetime(dataframe["retweet_date"], format='ISO8601')
    
    dataframe["time_difference"] = (dataframe["date"] - dataframe["retweet_date"]).dt.seconds
    
    features_df["average followers"] = dataframe.groupby("news_id")["follower_count"].mean()
    features_df["average follows"] = dataframe.groupby("news_id")["follows_count"].mean()
    features_df["repost total"] = dataframe.groupby("news_id")["type"].value_counts().unstack()["repost"].fillna(0).astype(int)
    features_df["post total"] = dataframe.groupby("news_id")["type"].value_counts().unstack()["post"].fillna(0).astype(int)
    features_df["repost percentage"] = features_df["repost total"] / (features_df["repost total"] + features_df["post total"])
    features_df["average retweet"] = dataframe.groupby("news_id")["repost_count"] / \
                                (features_df["repost total"] + features_df["post total"])
    features_df["average favorite"] = dataframe.groupby("news_id")["like_count"].mean()
    features_df["label"] = label # 0 for fake news, 1 for real news
    features_df["news lifetime"] = (dataframe.groupby("news_id")["date"].max() \
                                    - dataframe.groupby("news_id")["date"].min()).dt.seconds
    
    features_df["nb users 10 hours"] = dataframe[dataframe["date"] <= dataframe["date"].min() \
                                        + pd.Timedelta(hours=10)].groupby("news_id")["user_name"].nunique().fillna(0).astype(int)

    features_df["average time difference"] = features_df["average time difference"] = dataframe.groupby("news_id")["time_difference"].mean().fillna(0).astype(int)
    
    # Filter rows where date is within 1 hour of the minimum date
    filtered_df = dataframe[dataframe["date"] < dataframe["date"].min() + pd.Timedelta(hours=1)]

    # Group by "news_id" and count values in the "type" column
    type_counts = filtered_df.groupby("news_id")["type"].value_counts().unstack(fill_value=0)

    # Safely retrieve "repost" and "post" counts
    repost_count = type_counts.get("repost", pd.Series(0, index=type_counts.index))
    post_count = type_counts.get("post", pd.Series(0, index=type_counts.index))

    # Calculate retweet percentage for 1 hour
    features_df["retweet percentage 1 hour"] = (repost_count + post_count) / \
                (features_df["retweet total"] + features_df["tweet total"]).fillna(0).astype(int)
                                                
    return features_df

In [104]:
politifact_fake_features = get_features(politifact_fake_posts, 0)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (27, 2) + inhomogeneous part.

In [98]:
politifact_fake_features.head()

,average followers,average follows,retweet total,tweet total,retweet percentage,average retweet,average favorite,label,news lifetime,nb users 10 hours,average time difference,retweet percentage 1 hour
news_id,,,,,,,,,,,,
politifact13467,855.476190,647.428571,11,10,0.523810,7.733333,30.571429,0,60962,NaN,NaN,NaN
politifact13496,189.000000,930.000000,0,1,0.000000,7.733333,0.000000,0,0,NaN,NaN,NaN
politifact13515,942.181818,913.727273,1,10,0.090909,7.733333,0.818182,0,48567,NaN,NaN,NaN
politifact13687,514.000000,1026.000000,0,1,0.000000,7.733333,0.000000,0,0,NaN,NaN,NaN
politifact13731,541.409091,780.636364,2,20,0.090909,7.733333,0.272727,0,37965,NaN,NaN,NaN


In [102]:
politifact_fake_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, politifact13467 to politifact15606
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   average followers          27 non-null     float64
 1   average follows            27 non-null     float64
 2   retweet total              27 non-null     int32  
 3   tweet total                27 non-null     int32  
 4   retweet percentage         27 non-null     float64
 5   average retweet            27 non-null     float64
 6   average favorite           27 non-null     float64
 7   label                      27 non-null     int64  
 8   news lifetime              27 non-null     int32  
 9   nb users 10 hours          1 non-null      float64
 10  average time difference    0 non-null      float64
 11  retweet percentage 1 hour  1 non-null      float64
dtypes: float64(8), int32(3), int64(1)
memory usage: 3.5+ KB


In [45]:
politifact_real_reposts = fpu.add_reposts(dataset_politifact_real, 10)

 37%|███▋      | 453/1222 [06:51<11:38,  1.10it/s]  


KeyError: 'repostedBy'

In [ ]:
politifact_real_reposts.head()

In [ ]:
politifact_real_reposts.info()

In [ ]:
politifact_real_posts = pd.concat([dataset_politifact_real, politifact_real_reposts])

In [ ]:
politifact_fake_posts.to_csv("data/politifact_real_posts.csv", index=False)

In [ ]:
politifact_real_features = get_features(politifact_real_posts, 1)